In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [5]:
path = '../nothing/concat_csv.txt'
df1 = pd.read_csv(path, header=None)

In [6]:
df2 = pd.read_csv('../passing/concat_csv.txt', header=None)
df3 = pd.read_csv('../touching/concat_csv.txt', header=None)

In [7]:
df1

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,147.642678,135.382561,123.509545,112.023631,100.924817,90.213104,79.888492,69.950982,60.400572,51.237264,...,-21.156407,-22.340990,-23.543316,-24.763386,-26.001198,-27.256754,-28.530053,-29.821095,-31.129880,nothing
1,28.991800,28.031470,27.077141,26.128814,25.186488,24.250163,23.319839,22.395517,21.477195,20.564875,...,-16.725994,-17.997894,-19.305117,-20.647664,-22.025534,-23.438728,-24.887245,-26.371086,-27.890250,nothing
2,64.453502,60.934952,57.489579,54.117382,50.818362,47.592518,44.439850,41.360359,38.354044,35.420905,...,-32.325674,-29.069234,-25.608144,-21.942404,-18.072014,-13.996975,-9.717285,-5.232947,-0.543958,nothing
3,-29.786449,-28.017430,-26.288322,-24.599127,-22.949843,-21.340472,-19.771013,-18.241466,-16.751831,-15.302108,...,123.626018,130.954866,138.409995,145.991404,153.699094,161.533063,169.493314,177.579844,185.792655,nothing
4,-65.061676,-62.382144,-59.738291,-57.130118,-54.557625,-52.020810,-49.519676,-47.054220,-44.624444,-42.230348,...,-29.098930,-32.759111,-36.565923,-40.519366,-44.619440,-48.866145,-53.259481,-57.799447,-62.486044,nothing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,-64.550711,-59.820475,-55.237428,-50.801569,-46.512898,-42.371415,-38.377120,-34.530013,-30.830094,-27.277364,...,134.239036,146.583228,159.320585,172.451107,185.974794,199.891646,214.201662,228.904844,244.001190,nothing
135,-51.660396,-47.953623,-44.353883,-40.861175,-37.475499,-34.196856,-31.025245,-27.960667,-25.003122,-22.152609,...,-28.955737,-33.894337,-39.034730,-44.376916,-49.920894,-55.666665,-61.614229,-67.763585,-74.114734,nothing
136,44.168982,43.962546,43.709727,43.410526,43.064942,42.672975,42.234626,41.749894,41.218779,40.641281,...,-31.596096,-32.847683,-34.095651,-35.340002,-36.580736,-37.817852,-39.051350,-40.281230,-41.507492,nothing
137,61.064023,60.707563,60.291428,59.815617,59.280131,58.684969,58.030133,57.315620,56.541433,55.707570,...,-40.302733,-41.314401,-42.301154,-43.262992,-44.199917,-45.111927,-45.999022,-46.861203,-47.698469,nothing


In [8]:
df2

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,90.196797,83.271277,76.532319,69.979922,63.614088,57.434815,51.442104,45.635955,40.016368,34.583343,...,-4.774296,-5.284786,-5.810692,-6.352016,-6.908756,-7.480914,-8.068489,-8.671481,-9.289890,passing
1,-172.160782,-167.537075,-162.897174,-158.241081,-153.568795,-148.880316,-144.175644,-139.454779,-134.717722,-129.964472,...,-10.350072,-11.101567,-11.870206,-12.655990,-13.458918,-14.278991,-15.116209,-15.970572,-16.842080,passing
2,3.620683,5.659635,7.605508,9.458304,11.218022,12.884662,14.458225,15.938709,17.326116,18.620445,...,-3.056452,-3.427264,-3.810025,-4.204735,-4.611393,-5.030000,-5.460556,-5.903061,-6.357515,passing
3,6.964435,6.699542,6.437533,6.178407,5.922163,5.668803,5.418326,5.170732,4.926021,4.684193,...,-4.344704,-4.171207,-3.982308,-3.778005,-3.558298,-3.323188,-3.072674,-2.806757,-2.525436,passing
4,-5.429256,-4.641892,-3.882333,-3.150577,-2.446624,-1.770476,-1.122131,-0.501590,0.091148,0.656082,...,0.767861,2.209436,3.715923,5.287323,6.923635,8.624861,10.390999,12.222050,14.118014,passing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,-194.657148,-182.845154,-171.335061,-160.126868,-149.220577,-138.616186,-128.313696,-118.313108,-108.614420,-99.217633,...,-4.918040,-4.484764,-4.022437,-3.531061,-3.010635,-2.461160,-1.882634,-1.275059,-0.638433,passing
65,35.342241,33.838325,32.355578,30.894001,29.453594,28.034355,26.636286,25.259386,23.903655,22.569094,...,-39.131625,-40.286163,-41.427404,-42.555349,-43.669997,-44.771349,-45.859405,-46.934164,-47.995627,passing
66,6.570937,6.423542,6.274271,6.123122,5.970098,5.815196,5.658418,5.499763,5.339232,5.176824,...,-30.957767,-33.402140,-35.905937,-38.469158,-41.091804,-43.773875,-46.515370,-49.316290,-52.176635,passing
67,24.280693,22.699491,21.156629,19.652108,18.185928,16.758088,15.368589,14.017430,12.704613,11.430135,...,-27.636168,-31.966586,-36.467078,-41.137643,-45.978282,-50.988996,-56.169783,-61.520643,-67.041578,passing


In [9]:
df3

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,43.657116,42.729162,41.787269,40.831437,39.861665,38.877955,37.880305,36.868717,35.843189,34.803722,...,84.621239,89.210233,93.866032,98.588637,103.378048,108.234263,113.157284,118.147111,123.203743,touching
1,87.565599,80.173725,73.038394,66.159604,59.537356,53.171649,47.062485,41.209862,35.613781,30.274242,...,-55.180271,-65.078049,-75.396357,-86.135193,-97.294558,-108.874452,-120.874875,-133.295827,-146.137308,touching
2,222.941540,207.672104,192.833841,178.426752,164.450835,150.906092,137.792521,125.110124,112.858900,101.038849,...,-71.890037,-76.473673,-81.145017,-85.904069,-90.750830,-95.685298,-100.707475,-105.817360,-111.014953,touching
3,73.395845,67.712083,62.228165,56.944089,51.859857,46.975469,42.290923,37.806221,33.521362,29.436347,...,-48.142450,-51.307903,-54.551852,-57.874297,-61.275238,-64.754676,-68.312610,-71.949039,-75.663966,touching
4,103.531320,100.106637,96.701508,93.315933,89.949912,86.603445,83.276532,79.969173,76.681369,73.413118,...,-90.361183,-101.197859,-112.437793,-124.080987,-136.127440,-148.577152,-161.430123,-174.686353,-188.345842,touching
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,-94.931893,-91.829883,-88.728177,-85.626774,-82.525675,-79.424878,-76.324385,-73.224194,-70.124307,-67.024724,...,-19.559930,-20.578879,-21.611400,-22.657491,-23.717154,-24.790388,-25.877193,-26.977570,-28.091517,touching
80,37.064956,35.065559,33.107031,31.189372,29.312583,27.476663,25.681613,23.927432,22.214121,20.541679,...,164.835519,178.539467,192.652347,207.174161,222.104908,237.444589,253.193202,269.350749,285.917229,touching
81,37.967361,35.729716,33.542686,31.406269,29.320467,27.285280,25.300707,23.366748,21.483404,19.650674,...,-59.116263,-61.183185,-63.245485,-65.303163,-67.356218,-69.404652,-71.448464,-73.487653,-75.522220,touching
82,-12.999904,-10.434385,-7.964577,-5.590480,-3.312092,-1.129415,0.957552,2.948809,4.844355,6.644191,...,87.176809,95.604779,104.327466,113.344871,122.656994,132.263834,142.165392,152.361667,162.852659,touching


In [10]:
df = df1.append(df2)
df = df.append(df3)

In [11]:
df = df.reset_index(drop=True)

In [24]:
FOLD = 7

labelA =  df[df.iloc[:,-1] == 'nothing']
labelB =  df[df.iloc[:,-1] == 'passing']
labelC =  df[df.iloc[:,-1] == 'touching']

sfA = shuffle(labelA)
testA, trainA = sfA[:int(len(sfA) / FOLD)], sfA[int(len(sfA) / FOLD):]
sfB = shuffle(labelB)
testB, trainB = sfB[:int(len(sfB) / FOLD)], sfB[int(len(sfB) / FOLD):]
sfC = shuffle(labelC)
testC, trainC = sfC[:int(len(sfC) / FOLD)], sfC[int(len(sfC) / FOLD):]

train_set = pd.concat([trainA, trainB, trainC])
test_set = pd.concat([testA, testB, testC])
train_set = shuffle(train_set)
test_set = shuffle(test_set)

x_train, y_train = train_set.iloc[:, 0:-1], train_set.iloc[:, -1]
x_test, y_test = test_set.iloc[:, 0:-1], test_set.iloc[:, -1]

# # standard
# sc = StandardScaler()
# sc.fit(x_train)
# x_train_std = sc.transform(x_train)
# x_test_std = sc.transform(x_test)

# # LabelEncoder
# le = LabelEncoder()
# le.fit(LABELS)
# y_train = le.transform(y_train)
# y_test = le.transform(y_test)